In [408]:
import pandas as pd
import plotly.express as px
import re

In [409]:
risk_factor = "../data/Kaggle/target_tables/8_risk_factors"

In [410]:
df_Age = pd.read_csv(risk_factor + "/Age.csv")

In [411]:
df_Diabetes = pd.read_csv(risk_factor + "/Diabetes.csv")

In [412]:
df_overweight = pd.read_csv(risk_factor + "/Overweight or obese.csv")

In [413]:
df_Age.columns

Index(['Unnamed: 0', 'Date', 'Study', 'Study Link', 'Journal', 'Severe',
       'Severe lower bound', 'Severe upper bound', 'Severe p-value',
       'Severe Significant', 'Severe Adjusted', 'Severe Calculated',
       'Fatality', 'Fatality lower bound', 'Fatality upper bound',
       'Fatality p-value', 'Fatality Significant', 'Fatality Adjusted',
       'Fatality Calculated', 'Multivariate adjustment', 'Study Type',
       'Sample Size', 'Study Population', 'Added on', 'Critical only',
       'Discharged vs. death?'],
      dtype='object')

In [414]:
datasets = {
    'Age Data': df_Age,
    'Diabetes Data': df_Diabetes,
    'Overweight Data': df_overweight
}

In [415]:
def severe_Fatality_plot_districution(dataset):
    df_analyse_Severe = dataset.dropna(subset=["Severe"]).copy()
    df_analyse_Fatality = dataset.dropna(subset=["Fatality"]).copy()
    fig_severe = px.histogram(df_analyse_Severe, x="Severe_number", title="Distribution de la Sévérité", nbins=100)
    fig_fatality = px.histogram(df_analyse_Fatality, x="Fatality_number", title="Distribution de la Létalité", nbins=100)
    fig_severe.show()
    fig_fatality.show()

In [416]:

def sample_size_plot_distribution(dataset):
    df_analyse_sample_size = dataset.dropna(subset=["Sample_Size"]).copy()
    
    # Création du box plot avec une couleur verte
    fig_sample = px.box(df_analyse_sample_size, y="Sample_Size", title="Distribution de la Taille de l'Échantillon", 
                         color_discrete_sequence=["green"])
    
    fig_sample.show()


In [417]:
def rename_columns(dataset):
    dataset.columns = [col.replace(" ", "_") for col in dataset.columns]

In [418]:
def proportion_presence(dataset):

    # Compter le nombre de NaN et de valeurs non NaN
    nan_count = dataset["Severe"].isna().sum()
    non_nan_count = dataset["Severe"].notna().sum()

    # Créer un DataFrame pour la visualisation
    nan_data = pd.DataFrame({
        "Type": ["NaN", "Non NaN"],
        "Count": [nan_count, non_nan_count]
    })

    # Création d'un graphique en camembert
    fig = px.pie(nan_data, names="Type", values="Count", title=f"Proportion de NaN dans Severe")
    fig.show()



        # Compter le nombre de NaN et de valeurs non NaN
    nan_count = dataset["Fatality"].isna().sum()
    non_nan_count = dataset["Fatality"].notna().sum()

    # Créer un DataFrame pour la visualisation
    nan_data = pd.DataFrame({
        "Type": ["NaN", "Non NaN"],
        "Count": [nan_count, non_nan_count]
    })

    # Création d'un graphique en camembert
    fig = px.pie(nan_data, names="Type", values="Count", title=f"Proportion de NaN dans Fatality")
    fig.show()

In [419]:
def extract_sample_studies(dataset):
    def extract_values(sample_str):
        if isinstance(sample_str, str):
            # Remplacer les virgules pour éviter les erreurs de conversion
            sample_str = sample_str.replace(",", "")

            # Extraire tous les nombres présents dans la chaîne
            numbers = re.findall(r"\d+", sample_str)
            
            if len(numbers) == 2:  # Si on trouve deux nombres, le premier est "studies", le second "patients"
                return int(numbers[1]), int(numbers[0])
            elif len(numbers) == 1:  # Si un seul nombre est trouvé, on considère que c'est "patients"
                return int(numbers[0]), 0

        return None, 0  # Si ce n'est pas une string ou pas de nombre détecté

    # Appliquer la fonction à la colonne Sample_Size
    dataset[["Sample_Size", "Studies"]] = dataset["Sample_Size"].apply(lambda x: pd.Series(extract_values(x)))

    return dataset

In [420]:
# df_overweight_test = df_overweight.copy()
# rename_columns(df_overweight_test)
# test = extract_sample_studies(df_overweight_test)

In [421]:
# test[["Sample_Size"]].max()

In [422]:
for name, dataset in datasets.items():
    rename_columns(dataset)
    dataset = extract_sample_studies(dataset)
    dataset["Severe_number"] = dataset["Severe"].astype(str).apply(lambda x: float(re.search(r"\d+(\.\d+)?", x).group()) if re.search(r"\d+(\.\d+)?", x) else None)
    dataset["Fatality_number"] = dataset["Fatality"].astype(str).apply(lambda x: float(re.search(r"\d+(\.\d+)?", x).group()) if re.search(r"\d+(\.\d+)?", x) else None)
    dataset["severe_outside_bounds"] = ((dataset["Severe_number"] < dataset["Severe_lower_bound"]) | (dataset["Severe_number"] > dataset["Severe_upper_bound"])).astype(int)
    dataset["fatality_outside_bounds"] = ((dataset["Fatality_number"] < dataset["Fatality_lower_bound"]) | (dataset["Fatality_number"] > dataset["Fatality_upper_bound"])).astype(int)
    nb_severe_outlier = dataset[dataset["severe_outside_bounds"] == 1]["severe_outside_bounds"].count()
    nb_fatalitie_outlier = dataset[dataset["fatality_outside_bounds"] == 1]["fatality_outside_bounds"].count()
    if nb_severe_outlier != 0:
        print("Il y a un risque severe pour " + str(nb_severe_outlier) + " revues concernant " + str(name))
    if nb_fatalitie_outlier != 0:
        print("Il y a un risque létal pour "+ str(nb_fatalitie_outlier) + " revues concernant " + str(name))
    severe_Fatality_plot_districution(dataset)
    sample_size_plot_distribution(dataset)
    proportion_presence(dataset)
    
df_Age = datasets["Age Data"]
df_Diabetes = datasets["Diabetes Data"]
df_overweight = datasets["Overweight Data"]

Il y a un risque létal pour 6 revues concernant Diabetes Data


Il y a un risque létal pour 2 revues concernant Overweight Data
